<a href="https://colab.research.google.com/github/Rajsoni03/Covid-X-ray-Dataset/blob/main/Mini_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [ ]:
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Input, Dense, MaxPooling2D, Conv2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import LearningRateScheduler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import shutil
import time
import cv2
import os

print('Library Importing Complete')

Library Importing Complete


# Importing Data

In [ ]:
# Copy data from Drive to Local Storage
shutil.copytree('/content/drive/MyDrive/AI Colab/Final Covid Diagnosis/data', '/content/data', copy_function = shutil.copy)

'/content/data'

In [ ]:
!git clone https://github.com/Rajsoni03/MyData.git

Cloning into 'MyData'...
remote: Enumerating objects: 7250, done.
remote: Total 7250 (delta 0), reused 0 (delta 0), pack-reused 7250
Receiving objects: 100% (7250/7250), 524.06 MiB | 45.03 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (7261/7261), done.


In [ ]:
os.mkdir('/content/data1')
os.mkdir('/content/data1/train')
os.mkdir('/content/data1/val')
os.mkdir('/content/data1/test')


os.mkdir('/content/data1/train/covid')
os.mkdir('/content/data1/train/normal')
os.mkdir('/content/data1/train/pneumonia')

os.mkdir('/content/data1/val/covid')
os.mkdir('/content/data1/val/normal')
os.mkdir('/content/data1/val/pneumonia')

os.mkdir('/content/data1/test/covid')
os.mkdir('/content/data1/test/normal')
os.mkdir('/content/data1/test/pneumonia')


In [ ]:
main_dir = "/content/data1"

def random_split(images, base_dir, class_name):
  length = len(images)
  images = images
  np.random.shuffle(images)
  
  no_of_train_end  = int(length * 0.8)
  no_of_val_end  = int(length * 0.9)

  print('Class:', class_name,'\nTrain Imgs:', no_of_train_end,'\nVal Imgs:',length-no_of_val_end,'\nTest Imgs:',  no_of_val_end-no_of_train_end, end='\n\n')

  for img in images[:no_of_train_end]:
    # print(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'train'), class_name), img))
    shutil.copy(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'train'), class_name), img))
  for img in images[no_of_val_end:]:
    # print(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'val'), class_name), img))
    shutil.copy(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir,  'val'), class_name), img))
  for img in images[no_of_train_end: no_of_val_end]:
    # print(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'test'), class_name), img))
    shutil.copy(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'test'), class_name), img))

In [ ]:
covid_img_src_dir = '/content/MyData/data/covid'
normal_img_src_dir = '/content/MyData/data/normal'
pneumonia_img_src_dir = '/content/MyData/data/pneumonia'

covid_imgs = os.listdir(covid_img_src_dir)
normal_imgs = os.listdir(normal_img_src_dir)
pneumonia_imgs = os.listdir(pneumonia_img_src_dir)

# copy images to covid and normal dir
random_split(covid_imgs, covid_img_src_dir, 'covid')
random_split(normal_imgs, normal_img_src_dir, 'normal')
random_split(pneumonia_imgs, pneumonia_img_src_dir, 'pneumonia')


Class: covid 
Train Imgs: 560 
Val Imgs: 70 
Test Imgs: 70

Class: normal 
Train Imgs: 1826 
Val Imgs: 229 
Test Imgs: 228

Class: pneumonia 
Train Imgs: 3418 
Val Imgs: 428 
Test Imgs: 427



In [ ]:
# initialize the training and Val data augmentation object
tarin_data_gen = ImageDataGenerator(rotation_range=15,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=0.1,
                                    zoom_range=.1,
                                    horizontal_flip=True,
                                    rescale=1./255)

train_data_gen = tarin_data_gen.flow_from_directory('/content/data/train',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

val_data_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = val_data_gen.flow_from_directory('/content/data/val',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical')

Found 2501 images belonging to 3 classes.
Found 317 images belonging to 3 classes.


#Creating Model

In [ ]:
input_X = Input(shape=(224, 224, 3))

X = Conv2D(16, (3, 3), activation='relu')(input_X)
X = MaxPooling2D((2, 2))(X)

X = Conv2D(32, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2))(X)

X = Conv2D(64, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2))(X)

X = AveragePooling2D((4,4))(X)
X = Flatten()(X)
X = Dense(512, activation='relu')(X)
X = Dense(128, activation='relu')(X)
X = Dense(3, activation='softmax')(X)

model = Model(inputs=input_X, outputs=X)

model.summary()

Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 26, 26, 64)      

In [ ]:
input_X = Input(shape=(224, 224, 3))

X = Conv2D(8, (3, 3), activation='relu')(input_X)
X = MaxPooling2D((2, 2))(X)

X1 = Conv2D(8, (3, 3), activation='relu')(X)
C1 = MaxPooling2D((2, 2))(X1)
X2 = Conv2D(8, (3, 3), activation='relu')(X)
C2 = MaxPooling2D((2, 2))(X2)
X3 = Conv2D(8, (3, 3), activation='relu')(X)
C3 = MaxPooling2D((2, 2))(X3)
X = Concatenate(axis=-1)([X1, X2, X3])

X = Conv2D(8, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2))(X)

X1 = Conv2D(16, (3, 3), activation='relu')(X)
C1 = MaxPooling2D((2, 2))(X1)
X2 = Conv2D(16, (3, 3), activation='relu')(X)
C2 = MaxPooling2D((2, 2))(X2)
X3 = Conv2D(16, (3, 3), activation='relu')(X)
C3 = MaxPooling2D((2, 2))(X3)
X = Concatenate(axis=-1)([X1, X2, X3])

X = Conv2D(8, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2))(X)

X1 = Conv2D(32, (3, 3), activation='relu')(X)
C1 = MaxPooling2D((2, 2))(X1)
X2 = Conv2D(32, (3, 3), activation='relu')(X)
C2 = MaxPooling2D((2, 2))(X2)
X3 = Conv2D(32, (3, 3), activation='relu')(X)
C3 = MaxPooling2D((2, 2))(X3)
X = Concatenate(axis=-1)([X1, X2, X3])

X = Conv2D(8, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2))(X)

X = AveragePooling2D((4,4))(X)
X = Flatten()(X)
X = Dropout(.8)(X)
X = Dense(1024, activation='relu')(X)
X = Dense(128, activation='relu')(X)
X = Dense(3, activation='softmax')(X)

model = Model(inputs=input_X, outputs=X)

model.summary()

Model: "functional_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 222, 222, 8)  224         input_10[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_57 (MaxPooling2D) (None, 111, 111, 8)  0           conv2d_58[0][0]                  
__________________________________________________________________________________________________
conv2d_59 (Conv2D)              (None, 109, 109, 8)  584         max_pooling2d_57[0][0]           
______________________________________________________________________________________

# Setup Hyperparameters

In [ ]:
Epochs = 100
BS = 32
LR = 0.001
decay = LR / Epochs

In [ ]:
# opt = Adam(lr=LR, decay=LR/Epochs)
model.compile(loss="categorical_crossentropy", optimizer='Adam', metrics=["accuracy"])

In [ ]:
# def lr_time_based_decay(epoch, lr):
#     return lr * 1 / (1 + decay * epoch)

In [87]:
# train the model
Mini_H = model.fit_generator(train_data_gen, 
                             steps_per_epoch=2501//BS,
                             epochs=Epochs,
                             validation_data=val_data_gen,
                            #  callbacks=[LearningRateScheduler(lr_time_based_decay, verbose=1)]
                             )

Epoch 1/100
78/78 [==============================] - 83s 1s/step - loss: 0.8950 - accuracy: 0.5294 - val_loss: 0.6519 - val_accuracy: 0.7192
Epoch 2/100
78/78 [==============================] - 81s 1s/step - loss: 0.6152 - accuracy: 0.7380 - val_loss: 0.4754 - val_accuracy: 0.8297
Epoch 3/100
78/78 [==============================] - 82s 1s/step - loss: 0.5165 - accuracy: 0.7712 - val_loss: 0.4601 - val_accuracy: 0.8139
Epoch 4/100
78/78 [==============================] - 82s 1s/step - loss: 0.4323 - accuracy: 0.8327 - val_loss: 0.3243 - val_accuracy: 0.8864
Epoch 5/100
78/78 [==============================] - 82s 1s/step - loss: 0.4066 - accuracy: 0.8457 - val_loss: 0.3438 - val_accuracy: 0.8644
Epoch 6/100
78/78 [==============================] - 82s 1s/step - loss: 0.3730 - accuracy: 0.8554 - val_loss: 0.3327 - val_accuracy: 0.8770
Epoch 7/100
78/78 [==============================] - 82s 1s/step - loss: 0.3656 - accuracy: 0.8659 - val_loss: 0.3256 - val_accuracy: 0.8770
Epoch 8/100
7

In [88]:
# Save the model
model.save('/content/drive/MyDrive/AI Colab/Final Covid Diagnosis/Mini_model.h5')
pd.DataFrame(Mini_H.history).to_csv('/content/drive/MyDrive/AI Colab/Final Covid Diagnosis/Mini_model.csv')

In [89]:
print('Done')

Done


In [ ]:
model.evaluate(train_data_gen)

79/79 [==============================] - 70s 891ms/step - loss: 0.2999 - accuracy: 0.8784


[0.2999034821987152, 0.8784486055374146]

In [ ]:
model.evaluate(val_data_gen)

10/10 [==============================] - 5s 545ms/step - loss: 0.3172 - accuracy: 0.8896


[0.31724685430526733, 0.8895899057388306]